In [8]:
import requests
from bs4 import BeautifulSoup
import select
import sqlite3

In [9]:
con = sqlite3.connect("bot3.db")

In [10]:
cur = con.cursor()
cur.connection

In [ ]:
cur.executescript("""
CREATE TABLE "table3" (
	"id"	INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
	"title"	TEXT NOT NULL,
	"content"	TEXT NOT NULL,
	"date"	REAL DEFAULT CURRENT_TIMESTAMP
);

CREATE TABLE "table4" (
	"id"	INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
	"table3_id"	INTEGER NOT NULL,
	"comment"	TEXT NOT NULL
);
""")

In [11]:
headers = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36"}
def download(method, url, param=None, data=None, timeout=1, maxretries=3):
    try:
        resp = requests.request(method, url,
                         params=param, data=None,
                         headers=headers)
        resp.raise_for_status()
    except requests.exceptions.HTTPError as e:
        if 500 <= e.response.status_code < 600\
            and maxretries > 0:
            print(maxretries)
            time.sleep(timeout)
            resp = download(method, url, param, data, timeout, maxretries-1)
            print("재시도")
        else:
            print(e.response.status_code)
            print(e.response.reason)       
    return resp

In [12]:
url = "http://www.ppomppu.co.kr/zboard/zboard.php"
param = {"id":"freeboard"}
html = download("get",url,param=param)
dom = BeautifulSoup(html.text, "html.parser")

In [13]:
seed = list()
queue = list()
seen = list()

In [14]:
seed.extend([requests.compat.urljoin(url,_["href"])
            for _ in dom.select("#page_list a")])

In [15]:
def parseURL(seed):
    html = download("get", seed)
    dom = BeautifulSoup(html.text, 'lxml')
    
    return [requests.compat.urljoin(seed,_.find_parent()["href"])
           for _ in dom.select("font.list_title")
           if _.find_parent() and _.find_parent().has_attr("href")]

In [16]:
def parseContent(url):
    html = download("get",url)
    dom = BeautifulSoup(html.text, "lxml")
    
    return {"title":
           dom.select_one("font.view_title2").text.strip(),
            "body":
            dom.select_one("td.board-contents").text.strip(),
            "comments":
            [_.text.strip()
            for _ in dom.select("div[id^=commentContent_]")]
           }

In [17]:
while seed:
    baseURL = seed.pop(0)
    seen.append(baseURL)
    linkList = parseURL(baseURL)
    
    for link in linkList:
        if link not in queue and link not in seed:
            queue.append(link)
            
    print("Queue: {0}, Seen: {1}".format(len(queue), len(seen)))

Queue: 30, Seen: 1
Queue: 60, Seen: 2
Queue: 90, Seen: 3
Queue: 120, Seen: 4
Queue: 150, Seen: 5
Queue: 180, Seen: 6
Queue: 210, Seen: 7
Queue: 240, Seen: 8
Queue: 270, Seen: 9
Queue: 300, Seen: 10


In [18]:
#contents = list()
while queue:
    baseURL = queue.pop(0)
    seen.append(baseURL)
    content = parseContent(baseURL)
    cur.execute("""
    INSERT INTO table3
    (title, content)
    VALUES (?,?)
    """, [content["title"], content["body"]])
    
    con.commit()
    
    cur.execute("""
    SELECT id
    FROM table3
    WHERE title=? AND content=?;
    """,[content["title"], content["body"]])
    
    table3_id = cur.fetchone()[0]
    
    for comment in content["comments"]:
        cur.execute("""
        INSERT INTO table4
        (table3_id, comment)
        VALUES(?,?)
        """,[table3_id, comment])
    con.commit()

KeyboardInterrupt: 

In [19]:
cur.execute("select * from sqlite_sequence")
cur.fetchall()

[('table3', 46), ('table4', 242)]

In [28]:
cur.close()
con.close()

클래스가 중첩될 때 원하는 값을 찾지 못할 수도있다.

In [6]:
requests.compat??